In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [3]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.19.0.0/24')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB"}])
vpc.wait_until_available()

In [4]:
# Crear routable
routetable_1 = vpc.create_route_table()
routetable_1.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB-RT"}])

[ec2.Tag(resource_id='rtb-07713b7a8bc9f31cf', key='Name', value='AWSCookbookVPC-206-RT-2')]

In [5]:
# Crear subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.19.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)

subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB-SN"}])

[ec2.Tag(resource_id='subnet-092ab1f7085360e1f', key='Name', value='AWSCookbook-206-SN-2')]

In [6]:
# Asociar la tabla de ruta con las subredes:
routetable_1.associate_with_subnet(SubnetId = subnet_1.id)

ec2.RouteTableAssociation(id='rtbassoc-015446f4c4cce2616')

In [9]:
# Crear grupo de seguridad
security_group_1 = vpc.create_security_group(
    GroupName='MySQL-DynamoDB-SG',
    Description='Allow SSH access',
    VpcId=vpc.id
)

In [12]:
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [13]:
# Create a role
role_name = 'MySQL-DynamoDB-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [14]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '554d79f1-3926-47de-8cc1-2122115a633a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '554d79f1-3926-47de-8cc1-2122115a633a',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 05 Oct 2022 01:06:43 GMT'},
  'RetryAttempts': 0}}

In [16]:
# Create an instance profile
instance_profile_name = 'Cookbook-SSM-Profile-206'
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)

In [17]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': '9a7f4eee-ed2c-4a98-82b0-9be9f753dba4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9a7f4eee-ed2c-4a98-82b0-9be9f753dba4',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Wed, 05 Oct 2022 01:07:12 GMT'},
  'RetryAttempts': 0}}

In [18]:
# Enable Dns in VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': '6624af0c-b26e-4f13-8aba-f519c51f684d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6624af0c-b26e-4f13-8aba-f519c51f684d',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Wed, 05 Oct 2022 01:07:25 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [70]:
# Create security group enable HHTPS:
security_group_https = vpc.create_security_group(
    GroupName='AWSCookbook-206-SG-HTTPS',
    Description='Allow HTTPS access',
    VpcId=vpc.id,
)

response = ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {'FromPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': vpc.cidr_block,
                    'Description': 'Allow HTTPS access'
                }
            ],
            'ToPort': 443
        }
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-06a04af61a4fe1b73',
   'GroupId': 'sg-0534e7ec73b99a91d',
   'GroupOwnerId': '329364475115',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '174.19.0.0/16',
   'Description': 'Allow HTTPS access'}],
 'ResponseMetadata': {'RequestId': '7c1c566b-64fd-4d71-a1db-f226f603c6e5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7c1c566b-64fd-4d71-a1db-f226f603c6e5',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '783',
   'date': 'Wed, 05 Oct 2022 01:53:14 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [71]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id, subnet_2.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id, subnet_2.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id, subnet_2.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [72]:
def vpc_endpoint_status(vpc_endpoint_id):
    vpc_endpoint = ec2_client.describe_vpc_endpoints(
        VpcEndpointIds=[vpc_endpoint_id]
    )
    return vpc_endpoint['VpcEndpoints'][0]['State']

In [73]:
print('Waiting for VPC endpoints to be available...')
while vpc_endpoint_status(vpc_endpoint_ssm['VpcEndpoint']['VpcEndpointId']) != 'available':
    time.sleep(20)
    print('SSM endpoint is not available yet...')   
print('SSM endpoint is available')

Waiting for VPC endpoints to be available...
SSM endpoint is not available yet...
SSM endpoint is not available yet...
SSM endpoint is not available yet...
SSM endpoint is not available yet...
SSM endpoint is available


In [22]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [11]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-Instance-206'
keypair = ec2.create_key_pair(KeyName=key_name)

In [25]:
# Create a EC2 instance:
instance_1 = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet_1.id,
    SecurityGroupIds=[security_group_1.id],
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance-1'
                },
            ]
        },
    ]
)

In [76]:
# Create another EC2 instance:
instance_2 = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet_2.id,
    SecurityGroupIds=[security_group_2.id],
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance-2'
                },
            ]
        },
    ]
)

In [33]:
# Cree una ruta de acceso a la red especificando las dos instancias de EC2 que ha desplegado y el puerto TCP 22:
response = ec2_client.create_network_insights_path(
    Source = instance_1[0].id,
    Destination = instance_2[0].id,
    Protocol = 'TCP',
    DestinationPort = 22
)

In [37]:
insights_path_id = response['NetworkInsightsPath']['NetworkInsightsPathId']
insights_path_id

'nip-0a8bdbefd91cbfa48'

In [38]:
# Start the network insights analysis between the two instances using the INSIGHTS_PATH_ID created in the previous step:
response = ec2_client.start_network_insights_analysis(
    NetworkInsightsPathId = insights_path_id
)

In [39]:
# Wait a few seconds until the analysis is done running and then view the results:
response = ec2_client.describe_network_insights_analyses(
    NetworkInsightsPathId = insights_path_id
)

In [46]:
# Save the results to json file:
with open('network_insights_analysis.json', 'w') as outfile:
    json.dump(response, outfile, indent=4, default=str)

En el archivo `network_insights_analysis.json` observe los campos *NetworkPathFound* y *ExplanationCode*:

In [ ]:
# Update the security group attached to instance 2. Add a rule to allow access from instance 1’s security group to TCP port 22 (SSH):
response = ec2_client.authorize_security_group_ingress(
    GroupId = security_group_2.id,
    IpPermissions = [
        {
            'FromPort': 22,
            'IpProtocol': 'tcp',
            'IpRanges': [

In [49]:
# Actualice el grupo de seguridad adjunto a la instancia 2 
# Añada una regla para permitir el acceso desde el grupo de seguridad de la instancia 1 al puerto TCP 22 (SSH):

response = ec2_client.authorize_security_group_ingress(
    GroupId=security_group_2.id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'UserIdGroupPairs': [
                {
                    'Description': 'Allow SSH access',
                    'GroupId': security_group_1.id
                }
            ]
        } 
    ]
)

In [50]:
# Vuelva a ejecutar el análisis de la red insights
# Utilice el mismo INSIGHTS_PATH_ID que utilizó anteriormente:

response = ec2_client.start_network_insights_analysis(
    NetworkInsightsPathId = insights_path_id
)

In [52]:
response = ec2_client.describe_network_insights_analyses(
    NetworkInsightsPathId = insights_path_id
)

In [53]:
# Rewrite the results to json file:
with open('network_insights_analysis.json', 'w') as outfile:
    json.dump(response, outfile, indent=4, default=str)

Ahora debería observar un campo `NetworkPathFound` con el valor `true`

#### Validación

In [80]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-0d17a46ea5a6f5c1f


In [ ]:
server=174.19.1.97
port=22
connect_timeout=5

timeout $connect_timeout bash -c "</dev/tcp/$server/$port"
if [ $? == 0 ];then
   echo "SSH Connection to $server over port $port is possible"
else
   echo "SSH connection to $server over port $port is not possible"
fi